In [1]:
from datetime import datetime
from hw3_mod import fit_FM
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import scipy



# define directories
base_dir = "/Users/yxiao/Documents/AM 2022/Quant Investing/ps6"

# load data
# 30 industries
Momentum = pd.read_excel(os.path.join(base_dir, "Problem_Set6.xls"),
                    sheet_name=0, skiprows=3)
Momentum = Momentum.rename(columns={"Unnamed: 0": "date"})
Momentum["date"] = pd.to_datetime(Momentum["date"], format="%Y%m")
Momentum = Momentum.set_index("date")
Momentum.index = Momentum.index.to_period("M").to_timestamp("M")

# FF4F
FF4F = pd.read_excel(os.path.join(base_dir, "Problem_Set6.xls"),
                    sheet_name=1, skiprows=2)
FF4F = FF4F.rename(columns={"Unnamed: 0": "date"})
FF4F["date"] = pd.to_datetime(FF4F["date"], format="%Y%m")
FF4F = FF4F.set_index("date")
FF4F.index = FF4F.index.to_period("M").to_timestamp("M")
FF4F = FF4F.rename(columns={"Mkt-RF": "Mkt_RF"})
FF4F_clear_na = FF4F.replace(-999, np.nan).dropna()

#Commodity 
Commodity = pd.read_excel(os.path.join(base_dir, "Problem_Set6.xls"),
                    sheet_name=2, skiprows=0)
Commodity = Commodity.rename(columns={"time": "date"})
Commodity["date"] = pd.to_datetime(Commodity["date"], format="%Y%m")
Commodity = Commodity.set_index("date")
Commodity.index = Commodity.index.to_period("M").to_timestamp("M")

In [2]:
# 125 Size-Past-return portfolios
spr = pd.read_excel(os.path.join(base_dir, "Problem_Set6.xls"),
                    sheet_name=3, skiprows=3)
spr = spr.replace(-99.99, np.nan)
spr = spr.replace(-999, np.nan)

In [3]:
# past 1 month return
spr_1m = spr.iloc[:,:26]
beme_c = spr_1m.iloc[0,:]
cols = ["c%s_%s" % (str(c_i).split(".")[0], c_j)
         for c_i, c_j in zip(spr_1m.columns, beme_c.values)]
cols = [c.replace(".", "_").replace("-", "_").replace(" ", "_")
         for c in cols]
spr_1m.columns = cols
spr_1m = spr_1m.iloc[1:,:]
spr_1m = spr_1m.rename(columns={"cSize_Past_1_month_return": "date"})
spr_1m["date"] = pd.to_datetime(spr_1m["date"], format="%Y%m")
spr_1m = spr_1m.set_index("date")
spr_1m.index = spr_1m.index.to_period("M").to_timestamp("M")
spr_1m = spr_1m.astype(float)

In [4]:
# past 2-12 month return
spr_12m = spr.iloc[:,26:52]
beme_c = spr_12m.iloc[0,:]
cols = ["c%s_%s" % (str(c_i).split(".")[0], c_j)
         for c_i, c_j in zip(spr_12m.columns, beme_c.values)]
cols = [c.replace(".", "_").replace("-", "_").replace(" ", "_")
         for c in cols]
spr_12m.columns = cols
spr_12m = spr_12m.iloc[1:,:]
spr_12m = spr_12m.rename(columns={"cSize_Past_2_12_month_return": "date"})
spr_12m["date"] = pd.to_datetime(spr_12m["date"], format="%Y%m")
spr_12m = spr_12m.set_index("date")
spr_12m.index = spr_12m.index.to_period("M").to_timestamp("M")
spr_12m = spr_12m.astype(float)

In [5]:
# past 13-60 month return
spr_60m = spr.iloc[:,52:78]
beme_c = spr_60m.iloc[0,:]
cols = ["c%s_%s" % (str(c_i).split(".")[0], c_j)
         for c_i, c_j in zip(spr_60m.columns, beme_c.values)]
cols = [c.replace(".", "_").replace("-", "_").replace(" ", "_")
         for c in cols]
spr_60m.columns = cols
spr_60m = spr_60m.iloc[1:,:]
spr_60m = spr_60m.rename(columns={"cSize_Past_13_60_month_return": "date"})
spr_60m["date"] = pd.to_datetime(spr_60m["date"], format="%Y%m")
spr_60m = spr_60m.set_index("date")
spr_60m.index = spr_60m.index.to_period("M").to_timestamp("M")
spr_60m = spr_60m.astype(float)

In [6]:
df_1m = pd.concat([FF4F, spr_1m], axis=1, join="inner")
df_1m = df_1m.replace(-999, np.nan).dropna()

df_12m = pd.concat([FF4F, spr_12m], axis=1, join="inner")
df_12m = df_12m.replace(-999, np.nan).dropna()

df_60m = pd.concat([FF4F, spr_60m], axis=1, join="inner")
df_60m = df_60m.replace(-999, np.nan).dropna()

In [7]:
def mom_strategy(n, Momentum, gap_period): #past n periods, based on the momentum dataframe, gap_period: apply to which following period strategy
    cumprod = (1 + Momentum/100).rolling(window = n).agg(lambda x :x.prod())/(1 + Momentum/100).rolling(window = gap_period).agg(lambda x :x.prod()) -1
    cumprod = cumprod.dropna() #drop the previous 12 months if it is rolling 12 months strategy
    index_of_comprod = cumprod.index
    cumprod_copy = cumprod
    cumprod_copy.reset_index(inplace = True, drop = True)
    Momentum_return = pd.DataFrame(columns = {'Return'})
    for i in range(len(cumprod)-1):
        cumprod_copy.sort_values(by = i ,ascending = True, axis=1,inplace = True) #Each period, sort the cumulative returns by ascending order
        short = Momentum.iloc[i+n][cumprod_copy.iloc[i,0:3].index].mean() #short the min 3 industry
        long = Momentum.iloc[i+n][cumprod_copy.iloc[i,-3:].index].mean() #long the max 3 industry 
        return_LMS = long - short
        Momentum_return = Momentum_return.append({'Return':return_LMS} ,ignore_index = True)
    Momentum_return.set_index(index_of_comprod[1:],inplace = True)
    return Momentum_return
    

In [8]:
def mom_strategy_commodity(n, Momentum, gap_period): #past n periods, based on the momentum dataframe, gap_period: apply to which following period strategy
    cumprod = (1 + Momentum/100).rolling(window = n).agg(lambda x :x.prod())/(1 + Momentum/100).rolling(window = gap_period).agg(lambda x :x.prod()) -1
    cumprod = cumprod.iloc[n-1:,:]
    index_of_comprod = cumprod.index
    cumprod_copy = cumprod
    cumprod_copy.reset_index(inplace = True, drop = True)
    Momentum_return = pd.DataFrame(columns = {'Return'})
    for i in range(len(cumprod)-1):
        cumprod_copy.sort_values(by = i ,ascending = True, axis=1,inplace = True) #Each period, sort the cumulative returns by ascending order
        cumprod_copy_copy = cumprod_copy.iloc[i,:].dropna()
        short = Momentum.iloc[i+n][cumprod_copy_copy.iloc[0:3].index].mean() #short the min 3 industry
        long = Momentum.iloc[i+n][cumprod_copy_copy.iloc[-3:].index].mean() #long the max 3 industry 
        return_LMS = long - short
        Momentum_return = Momentum_return.append({'Return':return_LMS} ,ignore_index = True)
    Momentum_return.set_index(index_of_comprod[1:],inplace = True)
    return Momentum_return
    

In [9]:
def fit_FM_beta(portfolio_df, factor_df):
    """fits FM procedure to provided portfolios and factors

    Parameters
    ----------
    portfolio_df : pandas DataFrame
        each column corresponds to a portfolio return to test
        index corresponds to dates
        (should align with factor_df format)
    factor_df : pandas DataFrame
        each column corresponds to a factor
        index corresponds to dates
        (should align with portfolio_df format)

    Returns
    -------
    tuple containing DataFrames:
        time-series betas
        time-series gamma est
        time-series aggregate gamma
    """

    # get labels and merge
    pf_lab = list(portfolio_df.columns)
    factor_lab = list(factor_df.columns)
    df = portfolio_df.merge(factor_df, right_index=True,
                            left_index=True)

    # fit ts betas
    beta_l = []
    for c in pf_lab:
        df_c = df[[c] + factor_lab]
        df_c = df_c.dropna()
        formula = "%s ~ %s" % (c, " + ".join(factor_lab))
        mod = smf.ols(formula, data=df_c)
        beta_l.append(mod.fit().params)
    beta_df = pd.DataFrame(beta_l, index=pf_lab)
    return beta_df, mod.fit().summary()


In [10]:
def FF3F_test(FF4F,Momentum_1month_1month,strategy_name): #FF4F refers to the factor dataframe, Momentum_1month_1month refers to the momentum return, strategy name is used to print out the outcome.
    FF3F_1m1m = pd.concat([Momentum_1month_1month,FF4F[['SMB','HML','Mkt_RF']]],axis = 1).dropna() #Keep the overlap date of the mom strategy and FF4F
    FF3F_1m1m_reg = fit_FM_beta(FF3F_1m1m[['Return']], FF3F_1m1m[['SMB','HML','Mkt_RF']])
    Alpha_1m1m = FF3F_1m1m_reg[0].iloc[0,0]
    t_stats_temp_1m1m = FF3F_1m1m_reg[1].tables[1].as_html()
    t_stats_Alpha_1m1m = pd.read_html(t_stats_temp_1m1m, header = 0, index_col = 0)[0].iloc[0,2]
    if np.abs(t_stats_Alpha_1m1m) > 1.96:
        print("The Alpha for " + str(strategy_name) + " is: " + str(Alpha_1m1m) + ". The t_stats for " + str(strategy_name) + " is: " + 
          str(t_stats_Alpha_1m1m) + ". The 3 factor cannot fully price the strategy.")
    else:
        print("The Alpha for " + str(strategy_name) + " is: " + str(Alpha_1m1m) + ". The t_stats for " + str(strategy_name) + " is: " + 
          str(t_stats_Alpha_1m1m) + ". The 3 factor can price the strategy.")
    return

In [11]:
def FF4F_test(FF4F_clear_na,Momentum_1month_1month,strategy_name): #FF4F refers to the factor dataframe, Momentum_1month_1month refers to the momentum return, strategy name is used to print out the outcome.
    FF4F_1m1m = pd.concat([Momentum_1month_1month,FF4F_clear_na[['SMB','HML','Mkt_RF','UMD']]],axis = 1).dropna() #Keep the overlap date of the mom strategy and FF4F
    FF4F_1m1m_reg = fit_FM_beta(FF4F_1m1m[['Return']], FF4F_1m1m[['SMB','HML','Mkt_RF','UMD']])
    Alpha_1m1m_4F = FF4F_1m1m_reg[0].iloc[0,0]
    t_stats_temp_1m1m_4F = FF4F_1m1m_reg[1].tables[1].as_html()
    t_stats_Alpha_1m1m_4F = pd.read_html(t_stats_temp_1m1m_4F, header = 0, index_col = 0)[0].iloc[0,2]
    if np.abs(t_stats_Alpha_1m1m_4F) > 1.96:
        print("The Alpha for " + str(strategy_name) + " is: " + str(Alpha_1m1m_4F) + ". The t_stats for " + str(strategy_name) + " is: " + 
          str(t_stats_Alpha_1m1m_4F) + ". The 4 factor cannot fully price the strategy.")
    else:
        print("The Alpha for " + str(strategy_name) + " is: " + str(Alpha_1m1m_4F) + ". The t_stats for " + str(strategy_name) + " is: " + 
          str(t_stats_Alpha_1m1m_4F) + ". The 4 factor can price the strategy.")
    return

In [12]:
def GRS_test(port_df, mkt, smb, hml, umd, rf):
  
    # fit TS regression
    params_l = []
    resid_l = []
    for c in port_df.columns:
        mod_df = pd.DataFrame({c: port_df[c] - rf,
                               "Mkt": mkt, 'SMB':smb, 'HML':hml, 'UMD':umd})
        mod = smf.ols("%s ~ Mkt + SMB + HML + UMD" % c, data=mod_df)
        fit = mod.fit()
        params = fit.params
        params.index = ["Intercept_coef", "Mkt_coef", "SMB_coef", "HML_coef", "UMD_coef"]
        tvalues = fit.tvalues
        tvalues.index = ["Intercept_tstat", "Mkt_tstat", "SMB_tstat", "HML_tstat", "UMD_tstat"]
        params_l.append(pd.concat([params, tvalues]))
        resid = fit.resid
        resid.name = c
        resid_l.append(resid)
    params = pd.concat(params_l, axis=1).T
    params.index = port_df.columns
    resid = pd.concat(resid_l, axis=1)

    # calculate GRS test stat
    T, N = port_df.shape
    L = 1
    mkt_mn = mkt.mean()
    mkt_var = mkt.var()
    Sigma = resid.T.dot(resid) / (T-L-1)
    alpha = params["Intercept_coef"]
    grs = alpha.T.dot(np.linalg.inv(Sigma)).dot(alpha)
    grs /= (1 + mkt_mn ** 2 / mkt_var)
    grs *= ((T/N) * (T-N-L) / (T-L-1))
    grs_pval = 1. - scipy.stats.f.cdf(grs, N, T-N-L)

    return params, grs, grs_pval

In [13]:
#Problem a
Momentum_1month_1month = mom_strategy(1, Momentum, 0)
time_series_avg_a = Momentum_1month_1month.mean()
std_a = Momentum_1month_1month.std()
t_stat = (time_series_avg_a - 0)/(std_a/np.sqrt(len(Momentum_1month_1month)))
Rf_a = FF4F.loc[Momentum_1month_1month.index]
Sharpe_table = pd.concat([Momentum_1month_1month,Rf_a['RF']],axis =1)
Sharpe_ratio = (time_series_avg_a - Sharpe_table['RF'].mean())/std_a
print("The time-series average return is " + str(time_series_avg_a) + ". The standard deviation is " + str(std_a) + 
      ". The t-stat is " + str(t_stat) + ". The Sharpe ratio is " + str(Sharpe_ratio) + '.')

The time-series average return is Return    0.72725
dtype: float64. The standard deviation is Return    5.40972
dtype: float64. The t-stat is Return    4.393339
dtype: float64. The Sharpe ratio is Return    0.082288
dtype: float64.


In [14]:
#Problem b

import statsmodels.api as sm
def TS(y,x):
    x = sm.add_constant(x)
    model=sm.OLS(y,x)
    results = model.fit()
    residual = results.resid
    return results.params, residual

In [15]:
sample_means = np.mean(Momentum,axis=0)
right1 = np.var(sample_means)
print(right1)

0.012073985032643631


In [16]:
rf = FF4F["RF"][:-4]
a=[]
b=[]
for i in range(len(FF4F)-5):
    a.append(FF4F["Mkt_RF"][i])
    b.append(FF4F["Mkt_RF"][i+1])
covF = np.cov(b,a)[0][1]

beta = []
for i in range(len(Momentum.columns)):
    y = Momentum.iloc[:,i] - rf
    x = FF4F["Mkt_RF"][:-4]
    result = TS(y,x)
    para=result[0]
    beta.append(para[1])
    
beta_var = np.var(beta)

right2 = beta_var*covF
print(right2)

0.13046959793635046


In [51]:
covE = []
for i in range(len(Momentum.columns)):
    y = Momentum.iloc[:,i] - rf
    x = FF4F["Mkt_RF"][:-4]
    resid = TS(y,x)[1]
    for m in range(len(resid)-1):
        a.append(resid[m])
        b.append(resid[m+1])
    covE.append(np.cov(a,b)[0,1])
right3 = np.mean(covE)
print(right3)

0.6674549394128614


In [52]:
right = right1+right2+right3
left = time_series_avg_a
print(left,right)

Return    0.72725
dtype: float64 0.8099985223818554


## The third part contributes the most to the momentum profits.

In [19]:
#Problem c
Momentum_12month_1month = mom_strategy(12, Momentum, 0)
time_series_avg_c = Momentum_12month_1month.mean()
std_c = Momentum_12month_1month.std()
t_stat_c = (time_series_avg_c - 0)/(std_c/np.sqrt(len(Momentum_12month_1month)))
Rf_c = FF4F.loc[Momentum_12month_1month.index]
Sharpe_table_c = pd.concat([Momentum_12month_1month,Rf_c['RF']],axis =1)
Sharpe_ratio_c = (time_series_avg_c - Sharpe_table_c['RF'].mean())/std_c
print("The time-series average return is " + str(time_series_avg_c) + ". The standard deviation is " + str(std_c) + 
      ". The t-stat is " + str(t_stat_c) + ". The Sharpe ratio is " + str(Sharpe_ratio_c) + '.')

The time-series average return is Return    0.940725
dtype: float64. The standard deviation is Return    6.092562
dtype: float64. The t-stat is Return    5.019961
dtype: float64. The Sharpe ratio is Return    0.108089
dtype: float64.


In [20]:
#Problem d
Momentum_12month_1month_gap = mom_strategy(12, Momentum, 1)
time_series_avg_d = Momentum_12month_1month_gap.mean()
std_d = Momentum_12month_1month_gap.std()
t_stat_d = (time_series_avg_d - 0)/(std_d/np.sqrt(len(Momentum_12month_1month_gap)))
Rf_d = FF4F.loc[Momentum_12month_1month_gap.index]
Sharpe_table_d = pd.concat([Momentum_12month_1month_gap,Rf_d['RF']],axis =1)
Sharpe_ratio_d = (time_series_avg_d - Sharpe_table_d['RF'].mean())/std_d
print("The time-series average return is " + str(time_series_avg_d) + ". The standard deviation is " + str(std_d) + 
      ". The t-stat is " + str(t_stat_d) + ". The Sharpe ratio is " + str(Sharpe_ratio_d) + '.')

The time-series average return is Return    0.926402
dtype: float64. The standard deviation is Return    5.965484
dtype: float64. The t-stat is Return    5.048835
dtype: float64. The Sharpe ratio is Return    0.107991
dtype: float64.


In [21]:
#Problem e
#1m_1m strategy
FF3F_test(FF4F,Momentum_1month_1month,"1month 1month momentum strategy")

The Alpha for 1month 1month momentum strategy is: 0.7291717455095632. The t_stats for 1month 1month momentum strategy is: 4.373. The 3 factor cannot fully price the strategy.


In [22]:
#12m_1m strategy
FF3F_test(FF4F,Momentum_12month_1month,"12month 1month momentum strategy")

The Alpha for 12month 1month momentum strategy is: 1.1914123342172047. The t_stats for 12month 1month momentum strategy is: 6.547000000000001. The 3 factor cannot fully price the strategy.


In [23]:
#12m_1m_gap strategy
FF3F_test(FF4F,Momentum_12month_1month_gap,"12month 1month skipping momentum strategy")

The Alpha for 12month 1month skipping momentum strategy is: 1.168813528492203. The t_stats for 12month 1month skipping momentum strategy is: 6.567. The 3 factor cannot fully price the strategy.


In [24]:
# Problem f
#1m_1m strategy
FF4F_test(FF4F_clear_na,Momentum_1month_1month,"1month 1month momentum strategy")

The Alpha for 1month 1month momentum strategy is: 0.5924416140978659. The t_stats for 1month 1month momentum strategy is: 3.465. The 4 factor cannot fully price the strategy.


In [25]:
#12m_1m strategy
FF4F_test(FF4F_clear_na,Momentum_12month_1month,"12month 1month momentum strategy")

The Alpha for 12month 1month momentum strategy is: 0.22533690926903022. The t_stats for 12month 1month momentum strategy is: 1.655. The 4 factor can price the strategy.


In [26]:
#12m_1m_gap strategy
FF4F_test(FF4F_clear_na,Momentum_12month_1month_gap,"12month 1month skipping momentum strategy")

The Alpha for 12month 1month skipping momentum strategy is: 0.21319064687788514. The t_stats for 12month 1month skipping momentum strategy is: 1.617. The 4 factor can price the strategy.


In [27]:
# Problem g
#g(i)
com_momentum_1month_1month = mom_strategy_commodity(1, Commodity, 0)
com_momentum_1month_1month

,Return
date,
1970-02-28,0.036342
1970-03-31,0.020713
1970-04-30,0.036066
1970-05-31,0.002895
1970-06-30,0.044869
...,...
2015-08-31,0.011529
2015-09-30,-0.127176
2015-10-31,-0.077336


In [28]:
com_momentum_12month_1month = mom_strategy_commodity(12, Commodity, 0)
com_momentum_12month_1month

,Return
date,
1971-01-31,0.088998
1971-02-28,-0.003308
1971-03-31,-0.069181
1971-04-30,-0.030483
1971-05-31,0.019264
...,...
2015-08-31,-0.046325
2015-09-30,0.022521
2015-10-31,0.034670


In [29]:
com_momentum_12month_1month_gap = mom_strategy_commodity(12, Commodity, 1)
com_momentum_12month_1month_gap

,Return
date,
1971-01-31,0.088998
1971-02-28,-0.003308
1971-03-31,-0.069181
1971-04-30,-0.030483
1971-05-31,0.026990
...,...
2015-08-31,-0.008920
2015-09-30,0.041359
2015-10-31,-0.004309


## This matters. As more types of commodities adding in, it is hard to test if the momentum is casued by the characteristics of the fixed types of commodities. 
## For example, if we are going to test the mean-reversion explanation, this way of dealing data might add distractions to the dataset.

In [30]:
#g(ii)
# 3 Factor Model
#1m_1m strategy_commodity
FF3F_test(FF4F,com_momentum_1month_1month,"1month 1month momentum strategy")

The Alpha for 1month 1month momentum strategy is: 0.017411657603063527. The t_stats for 1month 1month momentum strategy is: 4.236000000000001. The 3 factor cannot fully price the strategy.


In [31]:
#12m_1m strategy_commodity
FF3F_test(FF4F,com_momentum_12month_1month,"12month 1month momentum strategy")

The Alpha for 12month 1month momentum strategy is: 0.010819916089178289. The t_stats for 12month 1month momentum strategy is: 2.697. The 3 factor cannot fully price the strategy.


In [32]:
#12m_1m_gap strategy_commodity
FF3F_test(FF4F,com_momentum_12month_1month_gap,"12month 1month skipping momentum strategy")

The Alpha for 12month 1month skipping momentum strategy is: 0.01012097679047665. The t_stats for 12month 1month skipping momentum strategy is: 2.583. The 3 factor cannot fully price the strategy.


In [33]:
# 4 Factor Model
#1m_1m strategy_commodity
FF4F_test(FF4F_clear_na,com_momentum_1month_1month,"1month 1month momentum strategy")

The Alpha for 1month 1month momentum strategy is: 0.015633388287010296. The t_stats for 1month 1month momentum strategy is: 3.733. The 4 factor cannot fully price the strategy.


In [34]:
#12m_1m strategy_commodity
FF4F_test(FF4F_clear_na,com_momentum_12month_1month,"12month 1month momentum strategy")

The Alpha for 12month 1month momentum strategy is: 0.006469396613087893. The t_stats for 12month 1month momentum strategy is: 1.614. The 4 factor can price the strategy.


In [35]:
#12m_1m_gap strategy_commodity
FF4F_test(FF4F_clear_na,com_momentum_12month_1month_gap,"12month 1month skipping momentum strategy")

The Alpha for 12month 1month skipping momentum strategy is: 0.0066676888868744955. The t_stats for 12month 1month skipping momentum strategy is: 1.689. The 4 factor can price the strategy.


In [36]:
#g(iii)
#1,1 IND MOM with 1,1 COM MOM
pd.concat([Momentum_1month_1month,com_momentum_1month_1month], axis = 1).dropna().corr().iloc[0,1]

0.13370707654083522

In [37]:
#12,1 IND MOM with 12,1 COM MOM
pd.concat([Momentum_12month_1month,com_momentum_12month_1month], axis = 1).dropna().corr().iloc[0,1]

0.1920289821928069

In [38]:
#12,1_gap IND MOM with 12,1_gap COM MOM
pd.concat([Momentum_12month_1month_gap,com_momentum_12month_1month_gap], axis = 1).dropna().corr().iloc[0,1]

0.1616323492260599

In [39]:
FF3F_12m1m_temp = pd.concat([Momentum_12month_1month,FF4F[['SMB','HML','Mkt_RF']]],axis = 1).dropna()
com_momentum_12month_1month_rename = com_momentum_12month_1month.rename(columns = {"Return": "COM_MOM"})
FF3F_12m1m = pd.concat([FF3F_12m1m_temp,com_momentum_12month_1month_rename],axis = 1).dropna()

In [40]:
#g(iv)
FF3F_12m1m_temp = pd.concat([Momentum_12month_1month,FF4F[['SMB','HML','Mkt_RF']]],axis = 1).dropna()
com_momentum_12month_1month_rename = com_momentum_12month_1month.rename(columns = {"Return": "COM_MOM"})
FF3F_12m1m = pd.concat([FF3F_12m1m_temp,com_momentum_12month_1month_rename],axis = 1).dropna()
FF3F_12m1m_reg = fit_FM_beta(FF3F_12m1m[['Return']], FF3F_12m1m[['SMB','HML','Mkt_RF','COM_MOM']])
Alpha_12m1m = FF3F_12m1m_reg[0].iloc[0,0]
t_stats_temp_12m1m = FF3F_12m1m_reg[1].tables[1].as_html()
t_stats_Alpha_12m1m = pd.read_html(t_stats_temp_12m1m, header = 0, index_col = 0)[0].iloc[0,2]
if np.abs(t_stats_Alpha_12m1m) > 1.96:
    print("The Alpha for 12month 1 month momemtum strategy is: " + str(Alpha_12m1m) + ". The t_stats for 12month 1 month momemtum strategy is: " + 
          str(t_stats_Alpha_12m1m) + ". The 3 factors with Commodity momentum cannot fully price the strategy.")
else:
    print("The Alpha for 12month 1 month momemtum strategy is: " + str(Alpha_12m1m) + ". The t_stats for 12month 1 month momemtum strategy is: " + 
          str(t_stats_Alpha_12m1m) + ".The 3 factors with Commodity momentum can price the strategy.")

The Alpha for 12month 1 month momemtum strategy is: 1.0047670108088367. The t_stats for 12month 1 month momemtum strategy is: 3.5439999999999996. The 3 factors with Commodity momentum cannot fully price the strategy.


In [41]:
#g(v)
FF3F_12m1m_reg = fit_FM_beta(FF3F_12m1m[['COM_MOM']], FF3F_12m1m[['SMB','HML','Mkt_RF','Return']])
Alpha_12m1m = FF3F_12m1m_reg[0].iloc[0,0]
t_stats_temp_12m1m = FF3F_12m1m_reg[1].tables[1].as_html()
t_stats_Alpha_12m1m = pd.read_html(t_stats_temp_12m1m, header = 0, index_col = 0)[0].iloc[0,2]
if np.abs(t_stats_Alpha_12m1m) > 1.96:
    print("The Alpha for 12month 1 month momemtum strategy is: " + str(Alpha_12m1m) + ". The t_stats for 12month 1 month momemtum strategy is: " + 
          str(t_stats_Alpha_12m1m) + ". The 3 factors with Industry momentum cannot fully price the strategy.")
else:
    print("The Alpha for 12month 1 month momemtum strategy is: " + str(Alpha_12m1m) + ". The t_stats for 12month 1 month momemtum strategy is: " + 
          str(t_stats_Alpha_12m1m) + ".The 3 factors with Industry momentum can price the strategy.")

The Alpha for 12month 1 month momemtum strategy is: 0.007572247908321237. The t_stats for 12month 1 month momemtum strategy is: 1.881.The 3 factors with Industry momentum can price the strategy.


g(vi)
## When adding commodity momentum to the 3 factors, the result still cannot price the industry momentum strategy. However, when doing the reverse by adding the industry momentum to the 3 factors, it succeeds in pricing the commodity momentum. 
## This outcome is quite surprising. Only either side takes effect. What's more, when considering their correlation, it is as low as about 0.19. Given that we conducting the 4 factor regession with the UMD" factor, one explanation would be that the industry momentum is closer to the "UMD" factor than the commondity momentum.

In [42]:
# Problem h
params_1m, grs_1m, grs_pval_1m = GRS_test(df_1m.iloc[:,5:31], df_1m.iloc[:,0], df_1m.iloc[:,1], df_1m.iloc[:,2], df_1m.iloc[:,4], df_1m.iloc[:,3])
grs_1m, grs_pval_1m

(10.113059752792768, 1.1102230246251565e-16)

In [43]:
params_12m, grs_12m, grs_pval_12m = GRS_test(df_12m.iloc[:,5:31], df_12m.iloc[:,0], df_12m.iloc[:,1], df_12m.iloc[:,2], df_12m.iloc[:,4], df_12m.iloc[:,3])
grs_12m, grs_pval_12m

(2.390466479091828, 0.00015562616612363467)

In [44]:
params_60m, grs_60m, grs_pval_60m = GRS_test(df_60m.iloc[:,5:31], df_60m.iloc[:,0], df_60m.iloc[:,1], df_60m.iloc[:,2], df_60m.iloc[:,4], df_60m.iloc[:,3])
grs_60m, grs_pval_60m

(2.079334190861083, 0.0014983175421063866)

Which sets of 25 portfolios are explained by the model and which aren’t? Which set of portfolios does the model have the most difficult time explaining? Which results are the most surprising?

## All sets are explained well by the model under 5% confidence level. However, only short-term (1 month) portfolios and intermediate-term (12-month) portfolios are explained well by the model under 1% confidence level. The model have most difficult explaining the long-term (60-month) portfolios.

## Momentum strategy by itself can only yield relatively low absolute return. According to our results, Fama French 3 factor model cannot price the momentum strategy well; while Fama French 4 factor model can price the momentum strategy better. Momentum strategy works the best for short-term 1-month past return sorted portfolios.